# Spurious Edge Filtering Demo (All Edges Named)
This notebook demonstrates the updated `read_edges_csv_and_filter_spurious` function from `src.filters`.

Unlike the previous demo, this version enriches **all** edges with page titles (names), not just the suspect edges. It loads an edge dataset, analyzes suspect edges, calculates a custom `final_weight`, and substitutes IDs with page titles for all edges for easier inspection and querying.

In [1]:
# Import Required Libraries and Setup Paths
import pandas as pd
import sys
import os

# Add project root to path
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)

from src import filters, edge_processing
INPUT_DATASET = '../data/out/SpotlightWeightSource_0102_0505_fullmatch.csv'  # Update if needed!
DB_PATH = '/home/vlr/Workspaces/WikipediaBiasProject/PyProject/data/out/graph_final.db'

In [2]:
# Load Edge Dataset Using New Function
import time
print(f"Loading and filtering: {INPUT_DATASET}")
start_time = time.time()
filtered_edges = edge_processing.read_edges_csv_and_filter_spurious(
    csv_path=INPUT_DATASET,
    db_path=DB_PATH
)
print(f"Done in {time.time() - start_time:.2f} seconds.")

# Filter out self edges immediately
# filtered_edges = filtered_edges[filtered_edges['source_wikidata_id'] != filtered_edges['target_wikidata_id']].copy()

Loading and filtering: ../data/out/SpotlightWeightSource_0102_0505_fullmatch.csv
Retrieving titles for 93,847 unique QIDs across 5 languages...
Computing shared names filter...
Done in 190.26 seconds.


In [3]:
# Analyze Suspect Edges
n_suspect = (filtered_edges['has_shared_word'] & (filtered_edges['z_score'] > 1.5)).sum()
n_total = len(filtered_edges)
print(f"Flagged {n_suspect:,} edges as suspect out of {n_total:,} ({(n_suspect/n_total)*100:.2f}%)")
suspect_df = filtered_edges[filtered_edges['has_shared_word'] & (filtered_edges['z_score'] > 1.5)].copy()
if n_suspect > 0:
    print("\nSample of flagged pseudo-self-loops:")
    display(suspect_df.head(20))
else:
    print("No suspect edges found.")

Flagged 33,194 edges as suspect out of 12,444,180 (0.27%)

Sample of flagged pseudo-self-loops:


,language_code,source_wikidata_id,target_wikidata_id,weight,fullmatch_count,has_shared_word,ordered_substring,z_score
398,fr,Q76065,Q76065,10.0,4,True,True,5.121453
438,en,Q2254738,Q856749,6.0,6,True,False,2.867242
1265,it,Q557092,Q312311,1.0,0,True,False,2.561738
1856,de,Q1337341,Q314241,1.0,1,True,False,3.712494
2196,en,Q1926870,Q57283,1.0,1,True,False,3.407918
2315,de,Q1926870,Q57283,1.0,1,True,False,3.407918
2375,fr,Q1926870,Q57283,1.0,1,True,False,3.407918
2495,it,Q1926870,Q57283,1.0,1,True,False,3.407918
2514,es,Q1926870,Q57283,1.0,1,True,False,3.407918
2825,de,Q274434,Q983501,2.0,2,True,False,2.146980


In [4]:
# Calculate Final Weight with Custom Logic
def calculate_final_weight(row):
    if row['z_score'] > 1.5 and row['has_shared_word']:
        if row['ordered_substring']:
            return 1.0
        else:
            return row['fullmatch_count']
    else:
        return row['weight']
filtered_edges['final_weight'] = filtered_edges.apply(calculate_final_weight, axis=1)
print("Sample with final_weight column:")
display(filtered_edges.head(20))

Sample with final_weight column:


,language_code,source_wikidata_id,target_wikidata_id,weight,fullmatch_count,has_shared_word,ordered_substring,z_score,final_weight
0,en,Q6882,Q53003,0.0,0,False,False,-0.108882,0.0
1,en,Q6882,Q5912,0.0,0,False,False,-0.108882,0.0
2,en,Q6882,Q212719,0.0,0,False,False,-0.108882,0.0
3,en,Q6882,Q514998,0.0,0,False,False,-0.108882,0.0
4,en,Q6882,Q210134,0.0,0,False,False,-0.108882,0.0
5,en,Q6882,Q55391,0.0,0,False,False,-0.108882,0.0
6,en,Q6882,Q1388518,0.0,1,False,False,-0.108882,0.0
7,en,Q6882,Q274143,0.0,0,False,False,-0.108882,0.0
8,en,Q6882,Q41406,0.0,0,False,False,-0.108882,0.0
9,en,Q6882,Q7371,0.0,0,False,False,-0.108882,0.0


In [5]:
# Retrieve Titles for All Edges
from src.modules.duckdb_handler import DuckDBHandler
# Collect all unique QIDs and language codes from the full filtered_edges DataFrame
unique_qids_all = list(set(filtered_edges['source_wikidata_id'].dropna()).union(set(filtered_edges['target_wikidata_id'].dropna())))
unique_langs_all = filtered_edges['language_code'].dropna().unique().tolist()
with DuckDBHandler(DB_PATH) as db:
    batch_size = 10000
    all_titles = []
    for i in range(0, len(unique_qids_all), batch_size):
        batch_qids = unique_qids_all[i : i + batch_size]
        batch_titles_df = db.get_titles_for_qids(qids=batch_qids, langs=unique_langs_all)
        all_titles.append(batch_titles_df)
    import pandas as pd
    if all_titles:
        titles_df = pd.concat(all_titles, ignore_index=True)
    else:
        titles_df = pd.DataFrame(columns=["wikidata_id", "language_code", "page_title"])

In [6]:
# Substitute IDs with Titles in All Edges (Enriched DataFrame)
enriched_edges = filtered_edges.copy()
enriched_edges = enriched_edges.merge(
    titles_df.rename(columns={'wikidata_id': 'source_wikidata_id', 'page_title': 'source_title'}),
    on=['source_wikidata_id', 'language_code'],
    how='left'
)
enriched_edges = enriched_edges.merge(
    titles_df.rename(columns={'wikidata_id': 'target_wikidata_id', 'page_title': 'target_title'}),
    on=['target_wikidata_id', 'language_code'],
    how='left'
)
enriched_edges['source_wikidata_id'] = enriched_edges['source_title'].fillna(enriched_edges['source_wikidata_id'])
enriched_edges['target_wikidata_id'] = enriched_edges['target_title'].fillna(enriched_edges['target_wikidata_id'])
enriched_edges = enriched_edges.drop(columns=['source_title', 'target_title'])

In [7]:
# Display Enriched DataFrame Sample
print(f"Sample of {len(enriched_edges)} enriched edge rows with substituted names:")
display(enriched_edges.sort_values(by='fullmatch_count', ascending=False).head(30))

Sample of 12444180 enriched edge rows with substituted names:


,language_code,source_wikidata_id,target_wikidata_id,weight,fullmatch_count,has_shared_word,ordered_substring,z_score,final_weight
5860998,en,Sting (wrestler),Sting (musician),0.0,556,True,True,-0.174887,0.0
4832580,fr,Dieudonné,Dieudonné,549.0,534,True,True,12.507963,1.0
6560500,en,Jesus,Jesus,26.0,466,True,True,7.961903,1.0
8584677,en,Cher,Cher,349.0,390,True,True,29.976255,1.0
7271967,en,Prince (musician),Prince (musician),11.0,362,False,True,2.660194,11.0
6925015,it,Flavio Claudio Giuliano,Giuliano (usurpatore),0.0,350,True,True,-0.313800,0.0
3078318,en,Saladin,Saladin,351.0,346,True,True,16.618532,1.0
10039531,en,Edge (wrestler),Edge (wrestler),7.0,343,True,True,2.121933,1.0
2126224,en,IU (entertainer),V (singer),0.0,342,False,False,-0.660011,0.0
9393401,de,Madonna (Künstlerin),Madonna (Künstlerin),1.0,332,True,True,0.019177,1.0


In [8]:
# Query Enriched DataFrame by Title
# Example: Query for a specific source or target title (replace as needed)
enriched_edges[(enriched_edges["source_wikidata_id"] == "Isabel Perón") & (enriched_edges["target_wikidata_id"] == "Juan Perón")]

,language_code,source_wikidata_id,target_wikidata_id,weight,fullmatch_count,has_shared_word,ordered_substring,z_score,final_weight
820925,en,Isabel Perón,Juan Perón,10.0,9,True,False,0.514669,10.0


In [9]:
# Corrected Final Weight Assignment Based on Multiple Conditions
def custom_final_weight(row):
    if row['has_shared_word'] and row['z_score'] > 1.5:
        if row['ordered_substring']:
            return 1
        else:
            return row['fullmatch_count']
    else:
        return row['weight']
enriched_edges['final_weight'] = enriched_edges.apply(custom_final_weight, axis=1)
print("Sample with corrected final_weight column:")
display(enriched_edges.head(20))

Sample with corrected final_weight column:


,language_code,source_wikidata_id,target_wikidata_id,weight,fullmatch_count,has_shared_word,ordered_substring,z_score,final_weight
0,en,James Joyce,Roberto Rossellini,0.0,0,False,False,-0.108882,0.0
1,en,James Joyce,Marcel Duchamp,0.0,0,False,False,-0.108882,0.0
2,en,James Joyce,Wilfred Owen,0.0,0,False,False,-0.108882,0.0
3,en,James Joyce,Richard Aldington,0.0,0,False,False,-0.108882,0.0
4,en,James Joyce,Louis Kahn,0.0,0,False,False,-0.108882,0.0
5,en,James Joyce,Robert Bresson,0.0,0,False,False,-0.108882,0.0
6,en,James Joyce,Oliver St. John Gogarty,0.0,1,False,False,-0.108882,0.0
7,en,James Joyce,Patrick Pearse,0.0,0,False,False,-0.108882,0.0
8,en,James Joyce,Edvard Munch,0.0,0,False,False,-0.108882,0.0
9,en,James Joyce,Federico Fellini,0.0,0,False,False,-0.108882,0.0


In [10]:
# Display edges where final_weight was changed to 1 or fullmatch_count (according to corrected logic)
# Edges where final_weight was set to 1 (ordered_substring case)
changed_ordered = enriched_edges[(enriched_edges['has_shared_word']) & (enriched_edges['z_score'] > 1.5) & (enriched_edges['ordered_substring']) & (enriched_edges['final_weight'] == 1)]
print(f"Number of edges with shared word, high z-score, and ordered substring (final_weight=1): {len(changed_ordered)}")
display(changed_ordered[['source_wikidata_id', 'target_wikidata_id', 'has_shared_word', 'z_score', 'ordered_substring', 'weight', 'fullmatch_count', 'final_weight']].head(20))

# Save changed_ordered to CSV
changed_ordered_path = '../data/out/changed_ordered_final_weight_1.csv'
changed_ordered.to_csv(changed_ordered_path, index=False)
print(f"Saved changed_ordered DataFrame to {changed_ordered_path}")

# Edges where final_weight was set to fullmatch_count (not ordered_substring case)
changed_not_ordered = enriched_edges[(enriched_edges['has_shared_word']) & (enriched_edges['z_score'] > 1.5) & (~enriched_edges['ordered_substring']) & (enriched_edges['final_weight'] == enriched_edges['fullmatch_count'])]
print(f"Number of edges with shared word, high z-score, and NOT ordered substring (final_weight=fullmatch_count): {len(changed_not_ordered)}")
display(changed_not_ordered[['source_wikidata_id', 'target_wikidata_id', 'has_shared_word', 'z_score', 'ordered_substring', 'weight', 'fullmatch_count', 'final_weight']].head(20))

# Save changed_not_ordered to CSV
changed_not_ordered_path = '../data/out/changed_not_ordered_final_weight_fullmatch_count.csv'
changed_not_ordered.to_csv(changed_not_ordered_path, index=False)
print(f"Saved changed_not_ordered DataFrame to {changed_not_ordered_path}")

Number of edges with shared word, high z-score, and ordered substring (final_weight=1): 4882


,source_wikidata_id,target_wikidata_id,has_shared_word,z_score,ordered_substring,weight,fullmatch_count,final_weight
398,Caspar Hedio,Caspar Hedio,True,5.121453,True,10.0,4,1.0
6998,Avishai Cohen (trumpeter),Avishai Cohen (bassist),True,2.138090,True,4.0,4,1.0
7002,Avishai Cohen (Trompeter),Avishai Cohen,True,3.741657,True,6.0,7,1.0
10192,Michail Aleksandrovič Romanov,Michail Aleksandrovič Romanov,True,1.742755,True,3.0,3,1.0
17780,Ralph Bakshi,Ralph Bakshi,True,8.884384,True,12.0,12,1.0
17931,Ephraim Katzir,Ephraim Katzir,True,7.723858,True,5.0,4,1.0
23463,Claudia Leitte,Claudia Leitte,True,2.437922,True,3.0,3,1.0
27062,Stephan VIII.,Stephan (II.),True,3.996195,True,1.0,10,1.0
28766,Ibn Idari,Ibn Idari,True,3.084543,True,1.0,0,1.0
30673,Mahinda Rajapaksa,Mahinda Rajapaksa,True,7.965733,True,144.0,34,1.0


Saved changed_ordered DataFrame to ../data/out/changed_ordered_final_weight_1.csv
Number of edges with shared word, high z-score, and NOT ordered substring (final_weight=fullmatch_count): 28312


,source_wikidata_id,target_wikidata_id,has_shared_word,z_score,ordered_substring,weight,fullmatch_count,final_weight
438,Lorenzo di Bicci,Bicci di Lorenzo,True,2.867242,False,6.0,6,6.0
1265,Giovanni d'Aragona (reggente di Trinacria),Pietro III d'Aragona,True,2.561738,False,1.0,0,0.0
1856,Ernesto Farías,José Ernesto Sosa,True,3.712494,False,1.0,1,1.0
2196,Michael Artin,Emil Artin,True,3.407918,False,1.0,1,1.0
2315,Michael Artin,Emil Artin,True,3.407918,False,1.0,1,1.0
2375,Michael Artin,Emil Artin,True,3.407918,False,1.0,1,1.0
2495,Michael Artin,Emil Artin,True,3.407918,False,1.0,1,1.0
2514,Michael Artin,Emil Artin,True,3.407918,False,1.0,1,1.0
2825,Ekaterina Guliyev,Ramil Guliyev,True,2.146980,False,2.0,2,2.0
2884,Marie Darrieussecq,Marie NDiaye,True,2.064900,False,2.0,2,2.0


Saved changed_not_ordered DataFrame to ../data/out/changed_not_ordered_final_weight_fullmatch_count.csv


In [11]:
# Save corrected CSV
import os
base_name = os.path.splitext(os.path.basename(INPUT_DATASET))[0]
output_path = f'../data/out/{base_name}_corrected.csv'

output = filtered_edges.drop(columns=['has_shared_word', 'ordered_substring', 'z_score', 'fullmatch_count', 'final_weight'], errors='ignore').copy()
output['weight'] = filtered_edges['final_weight']
output.to_csv(output_path, index=False)
print(f"Saved corrected edges to {output_path}")

Saved corrected edges to ../data/out/SpotlightWeightSource_0102_0505_fullmatch_corrected.csv
